<a href="https://colab.research.google.com/github/grnbatera/TFC/blob/main/TFC_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [281]:
!git clone https://github.com/grnbatera/TFC.git

fatal: destination path 'TFC' already exists and is not an empty directory.


In [282]:
#Instalando módulo i/o do tensorflow
!pip install -q tensorflow-io

In [283]:
#importando bibliotecas
%matplotlib inline  
import matplotlib.pyplot as plt #importa o pacote de plotagem
import numpy as np #pacote numpy faz manipulações numéricas
from IPython.display import Audio #importando ferramentas de audio
import librosa #importando biblioteca de ferramentas amplas de manipulação de audio
import librosa.display #importando ferramentas de spectograma do librosa
import librosa.feature
import pandas as pd
import math
import seaborn as sns
import matplotlib as mpl
import tensorflow as tf
import tensorflow_io as tfio
import scipy
from scipy import signal

In [284]:
mpl.rcParams['font.family'] = 'Roboto'
plt.rcParams['font.family'] = 'Roboto'
sns.set(font='Roboto',
        rc={
 'axes.axisbelow': False,
 'axes.edgecolor': 'lightgrey',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.labelcolor': 'dimgrey',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'black',
 'xtick.bottom': False,
 'xtick.color': 'dimgrey',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'dimgrey',
 'ytick.direction': 'out',
 'ytick.left': False,
 'ytick.right': False})
 
sns.set_context("notebook", rc={"font.size":20,
                                "axes.titlesize":24,
                                "axes.labelsize":18})
                             

#Condições de Contorno para o Violão

In [285]:
a1 = [0.8, -0.2];
g = .995;
b1 = g * (a1[0]+a1[1])
b1


0.5970000000000001

#Resposta ao Impulso de Corpo

In [286]:

fname1='/content/TFC/body1.wav'
#pegando os valores de sr e atribuindo audio para y
y1, fs=librosa.load(fname1, sr=None, mono=True) #lendo o fname, se sr diferente de None ele reamostra pra 22050 Hz


"\nfname1='/content/TFC/body1.wav'\n#pegando os valores de sr e atribuindo audio para y\ny1, fs=librosa.load(fname1, sr=None, mono=True) #lendo o fname, se sr diferente de None ele reamostra pra 22050 Hz\n"

In [287]:
y2=scipy.signal.unit_impulse(8192,)

In [288]:
y1

array([1., 0., 0., ..., 0., 0., 0.])

In [289]:
y1
Audio(data=y1,rate=fs)

In [290]:
fs

22050

#Parâmetros da waveguide inteira e da fracionária

In [291]:
f=440
phi=0.4275
D = math.floor(fs/f - phi);
d = fs/f - phi - D;

if d < 0.1:
  d = d + 1;
  D = D - 1;

In [292]:
D

49

In [293]:
d

0.6861363636363649

#Fitro Fracional

In [294]:
eta = (1-d) / (1+d);
b2 = [eta,1];
a2 = [1,eta];


#Filtro da Corda

In [295]:

num=np.convolve(b1,b2)
den=np.convolve(a1,a2)



In [296]:
den

array([ 0.8       , -0.05108505, -0.03722874])

In [297]:
num

array([0.11112778, 0.597     ])

In [298]:
k=D-len(den)
b=den
z=-num
m=np.zeros((k,))
a=np.hstack((b,m,z))
a

array([ 0.8       , -0.05108505, -0.03722874,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.11112778,
       -0.597     ])

In [299]:
b

array([ 0.8       , -0.05108505, -0.03722874])

In [300]:
DUR=3

#Síntese do Áudio

In [301]:
z1=np.zeros((DUR*fs-len(y1)),);
z1
x=np.hstack((y1,z1))

In [302]:
x

array([1., 0., 0., ..., 0., 0., 0.])

In [303]:
y = scipy.signal.lfilter(b,a,x);

#Normalizando y 
y = y/(max(abs(y))+.1);
y

array([9.09090909e-01, 0.00000000e+00, 0.00000000e+00, ...,
       2.35628512e-05, 2.35500319e-05, 2.35446562e-05])

In [304]:
Audio(data=y,rate=fs)